# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [8]:
# Function to call the model with error handling and retries.
import time
from openai import OpenAI, OpenAIError

def return_OAI(user_message):
    client = OpenAI(
        api_key=OPENAI_API_KEY,
    )
    context = [{'role': 'system', "content": user_message}]

    # Retry up to 3 times if the API fails
    for _ in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=context,
                temperature=0,
            )
            return response.choices[0].message.content
        except OpenAIError as e:
            print(f"Error: {e}, retrying...")
            time.sleep(2)

    return "Failed to connect to OpenAI API after retries."


In [9]:
# Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['Customers', 'Orders', 'Products'],  # Add table names here
        'definition': [
            'Stores customer information such as name, email, and address',  # Describe each table here
            'Records details of customer orders including order date, customer ID, and total amount',
            'Contains product details like name, category, and price'
        ]}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  Customers  Stores customer information such as name, emai...
1     Orders  Records details of customer orders including o...
2   Products  Contains product details like name, category, ...


In [10]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [11]:
print(text_tables)

Customers: Stores customer information such as name, email, and address
Orders: Records details of customer orders including order date, customer ID, and total amount
Products: Contains product details like name, category, and price


In [12]:
prompt_question_tables = """
Given the following tables and their content definitions,

### Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{question}
"""



In [13]:
pqt1 = prompt_question_tables.format(tables=text_tables, question="What are the most common diseases in the dataset?")


In [14]:
print(return_OAI(pqt1))

{
    "tables": ["Products"]
}


In [16]:
pqt3 = prompt_question_tables.format(tables=text_tables, 
                                     question="What is the average age of patients in the dataset?")


In [17]:
print(return_OAI(pqt3))

{
    "tables": ["Customers"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [18]:
###  Exercise: Modify for a new question

new_question = "List the names of customers who ordered products in the 'Electronics' category."

In [20]:
# Update the prompt for the new question
new_prompt = f"""
The following tables are available:
{text_tables}

Generate a SQL query to answer this question: {new_question}
"""

In [22]:
# Get the new SQL query
new_sql_query = return_OAI(new_prompt)
print("\nGenerated SQL query for new question:\n", new_sql_query)



Generated SQL query for new question:
 ```sql
SELECT DISTINCT c.name
FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id
JOIN OrderDetails od ON o.order_id = od.order_id
JOIN Products p ON od.product_id = p.product_id
WHERE p.category = 'Electronics';
```


### Report Summary

**Project Summary:**

We built a system that generates SQL queries based on table names and descriptions using GPT-3.5 Turbo.

1. Defined tables and descriptions.
2. Formed a prompt combining table details.
3. Sent the prompt to the OpenAI API.
4. Generated SQL queries to answer user questions.

**Results:**
- Successfully generated SQL queries for multiple user questions.
- Demonstrated flexibility by adapting to new queries without modifying the tables.

**Next Steps:**
- Improve handling of complex joins and conditions.
- Test with larger databases and more tables.
- Add error handling for API connection issues.
"""

